# Downloading model weights from wandb

In [2]:
import os, gc, sys, copy, pickle
from pathlib import Path
import glob
from tqdm.auto import tqdm
tqdm.pandas()

import math
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from joblib import Parallel, delayed
import multiprocessing as mp

import albumentations as A
import torch
import torch.nn as nn
import torch.optim as optim
import torch.cuda.amp as amp
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from torch.utils.data import WeightedRandomSampler
from sklearn.utils.class_weight import compute_class_weight

import timm

import cv2
cv2.setNumThreads(0)
import PIL
import pydicom
import warnings
warnings.filterwarnings("ignore")

In [3]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("WANDB_API_KEY")
wandb.login(key=secret_value_0)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
def seeding(SEED):
    np.random.seed(SEED)
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    torch.manual_seed(SEED)
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(SEED)
        torch.cuda.manual_seed_all(SEED)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
#     os.environ['TF_CUDNN_DETERMINISTIC'] = str(SEED)
#     tf.random.set_seed(SEED)
#     keras.utils.set_random_seed(seed=SEED)
    print('seeding done!!!')

def flush():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

In [5]:
CONFIG = dict(
    project_name = "RSNA-2024-Lumbar-Spine-Classification-Torch-RZoro",
    artifact_name = "rsnaEffNetModel",
    load_kernel = None,
    load_last = True,
    n_folds = 5,
    backbone = "efficientnet_b0.ra_in1k", # tf_efficientnetv2_s_in21ft1k
    img_size = 384,
    n_slice_per_c = 16,
    in_chans = 3,

    drop_rate = 0.,
    drop_rate_last = 0.3,
    drop_path_rate = 0.,
    p_mixup = 0.5,
    p_rand_order_v1 = 0.2,
    lr = 4e-4, # 1e-3, 8e-4, 5e-4, 4e-4

    out_dim = 3,
    epochs = 50,
    batch_size = 16,
#     patience = 7,
    device = torch.device("cuda:0") if torch.cuda.is_available() else "cpu",
    seed = 2024,
    log_wandb = True,
    with_clip = True,
)

CONFIG['patience'] = math.ceil(0.2 * CONFIG['epochs'])

seeding(CONFIG['seed'])

seeding done!!!


# Load Model

In [6]:
class TimmModel(nn.Module):
    def __init__(self, backbone, pretrained=False):
        super(TimmModel, self).__init__()

        self.encoder = timm.create_model(
            backbone,
            num_classes=CONFIG["out_dim"],
            features_only=False,
            drop_rate=CONFIG["drop_rate"],
            drop_path_rate=CONFIG["drop_path_rate"],
            pretrained=pretrained
        )

        if 'efficient' in backbone:
            hdim = self.encoder.conv_head.out_channels
            self.encoder.classifier = nn.Identity()
        elif 'convnext' in backbone:
            hdim = self.encoder.head.fc.in_features
            self.encoder.head.fc = nn.Identity()


        self.lstm = nn.LSTM(hdim, 256, num_layers=2, dropout=CONFIG["drop_rate"], bidirectional=True, batch_first=True)
        self.head = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.Dropout(CONFIG["drop_rate_last"]),
            nn.LeakyReLU(0.1),
            nn.Linear(256, CONFIG["out_dim"]),
        )

    def forward(self, x):
        feat = self.encoder(x)
        feat, _ = self.lstm(feat)
        feat = self.head(feat)
        return feat

# Cross validation

In [7]:
DATA_PATH = Path("/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification")
os.listdir(DATA_PATH)

['sample_submission.csv',
 'train_images',
 'train_series_descriptions.csv',
 'train.csv',
 'train_label_coordinates.csv',
 'test_series_descriptions.csv',
 'test_images']

In [8]:
train_main = pd.read_csv(DATA_PATH/"train.csv")
train_desc = pd.read_csv(DATA_PATH/"train_series_descriptions.csv")
train_label_coordinates = pd.read_csv(DATA_PATH/"train_label_coordinates.csv")

In [9]:
# Define function to reshape a single row of the DataFrame
def reshape_row(row):
    data = {'study_id': [], 'condition': [], 'level': [], 'severity': []}
    
    for column, value in row.items():
        if column not in ['study_id', 'series_id', 'instance_number', 'x', 'y', 'series_description']:
            parts = column.split('_')
            condition = ' '.join([word.capitalize() for word in parts[:-2]])
            level = parts[-2].capitalize() + '/' + parts[-1].capitalize()
            data['study_id'].append(row['study_id'])
            data['condition'].append(condition)
            data['level'].append(level)
            data['severity'].append(value)
    
    return pd.DataFrame(data)

# Reshape the DataFrame for all rows
new_train_df = pd.concat([reshape_row(row) for _, row in train_main.iterrows()], ignore_index=True)

# Display the first few rows of the reshaped dataframe
new_train_df.head(5)

,study_id,condition,level,severity
0,4003253,Spinal Canal Stenosis,L1/L2,Normal/Mild
1,4003253,Spinal Canal Stenosis,L2/L3,Normal/Mild
2,4003253,Spinal Canal Stenosis,L3/L4,Normal/Mild
3,4003253,Spinal Canal Stenosis,L4/L5,Normal/Mild
4,4003253,Spinal Canal Stenosis,L5/S1,Normal/Mild


In [10]:
# Merge the dataframes on the common columns
merged_df = pd.merge(new_train_df, train_label_coordinates, on=['study_id', 'condition', 'level'], how='inner')
final_merged_df = pd.merge(merged_df, train_desc, on=['series_id','study_id'], how='inner')

# Create the row_id column
final_merged_df['row_id'] = (
    final_merged_df['study_id'].astype(str) + '_' +
    final_merged_df['condition'].str.lower().str.replace(' ', '_') + '_' +
    final_merged_df['level'].str.lower().str.replace('/', '_')
)

# Create the image_path column
final_merged_df['image_path'] = (
    f'{str(DATA_PATH)}/train_images/' + 
    final_merged_df['study_id'].astype(str) + '/' +
    final_merged_df['series_id'].astype(str) + '/' +
    final_merged_df['instance_number'].astype(str) + '.dcm'
)

final_merged_df['severity'] = final_merged_df['severity'].map(
    {'Normal/Mild': 'normal_mild', 'Moderate': 'moderate', 'Severe': 'severe'}
)

train_data = final_merged_df.copy()
# Display the updated dataframe
train_data.head(5)

,study_id,condition,level,severity,series_id,instance_number,x,y,series_description,row_id,image_path
0,4003253,Spinal Canal Stenosis,L1/L2,normal_mild,702807833,8,322.831858,227.964602,Sagittal T2/STIR,4003253_spinal_canal_stenosis_l1_l2,/kaggle/input/rsna-2024-lumbar-spine-degenerat...
1,4003253,Spinal Canal Stenosis,L2/L3,normal_mild,702807833,8,320.571429,295.714286,Sagittal T2/STIR,4003253_spinal_canal_stenosis_l2_l3,/kaggle/input/rsna-2024-lumbar-spine-degenerat...
2,4003253,Spinal Canal Stenosis,L3/L4,normal_mild,702807833,8,323.030303,371.818182,Sagittal T2/STIR,4003253_spinal_canal_stenosis_l3_l4,/kaggle/input/rsna-2024-lumbar-spine-degenerat...
3,4003253,Spinal Canal Stenosis,L4/L5,normal_mild,702807833,8,335.292035,427.327434,Sagittal T2/STIR,4003253_spinal_canal_stenosis_l4_l5,/kaggle/input/rsna-2024-lumbar-spine-degenerat...
4,4003253,Spinal Canal Stenosis,L5/S1,normal_mild,702807833,8,353.415929,483.964602,Sagittal T2/STIR,4003253_spinal_canal_stenosis_l5_s1,/kaggle/input/rsna-2024-lumbar-spine-degenerat...


In [11]:
# Define a function to check if a path exists
def check_exists(path):
    return os.path.exists(path)

# Define a function to check if a study ID directory exists
def check_study_id(row):
    study_id = row['study_id']
    path = f'{str(DATA_PATH)}/train_images/{study_id}'
    return check_exists(path)

# Define a function to check if a series ID directory exists
def check_series_id(row):
    study_id = row['study_id']
    series_id = row['series_id']
    path = f'{str(DATA_PATH)}/train_images/{study_id}/{series_id}'
    return check_exists(path)

# Define a function to check if an image file exists
def check_image_exists(row):
    image_path = row['image_path']
    return check_exists(image_path)

# Apply the functions to the train_data dataframe
train_data['study_id_exists'] = train_data.progress_apply(check_study_id, axis=1)
train_data['series_id_exists'] = train_data.progress_apply(check_series_id, axis=1)
train_data['image_exists'] = train_data.progress_apply(check_image_exists, axis=1)

# Filter train_data
train_data = train_data[(train_data['study_id_exists']) & (train_data['series_id_exists']) & (train_data['image_exists'])]

  0%|          | 0/48692 [00:00<?, ?it/s]

  0%|          | 0/48692 [00:00<?, ?it/s]

  0%|          | 0/48692 [00:00<?, ?it/s]

In [12]:
label2id = {v: i for i, v in enumerate(train_data['severity'].unique())}
id2label = {v:k for k,v in label2id.items()}
train_data['target'] = train_data['severity'].map(label2id)
train_data = train_data.dropna(subset=['severity']).reset_index(drop=True)

# series2id = {v:i for i, v in enumerate(train_data['series_description'].unique().tolist())}
# id2series = {v:k for k,v in series2id.items()}
# train_data['series2id'] = train_desc['series_description'].map(series2id)
# train_data = train_data.dropna(subset=['series2id']).reset_index(drop=True)

In [13]:
def load_dicom(path):
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    data = data - np.min(data)
    if np.max(data) != 0:
        data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

In [14]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None, label_name='target'):
        self.dataframe = dataframe
        self.transform = transform
        self.label = dataframe.loc[:, label_name]

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        image_path = self.dataframe['image_path'][index]
        image = load_dicom(image_path)  # Define this function to load your DICOM images
        target = self.dataframe['target'][index]
        
        if self.transform:
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
            image = self.transform(image=image)['image']
            image = image.transpose(2, 0, 1).astype(np.float32) / 255.

        return image, torch.tensor(target).float()
    
    def get_labels(self):
        return self.label

In [15]:
def get_transforms(height, width):
    train_tsfm = A.Compose([
        # Geometric augmentations
#         A.Perspective(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
#         A.RandomRotate90(p=0.5),
        A.Rotate(-30, 30, p=0.5),
        
        A.Resize(height=height, width=width),
    ])
    
    valid_tsfm = A.Compose([
        A.Resize(height=height, width=width),
    ])
    return {"train": train_tsfm, "eval": valid_tsfm}

def get_dataloaders(data, cfg, split="train"):
    img_size = cfg['img_size']
    height, width = img_size, img_size
    tsfm = get_transforms(height=height, width=width)
    if split == 'train':
        tr_tsfm = tsfm['train']
        ds = CustomDataset(data, transform=tr_tsfm)
        labels = ds.get_labels()
#         class_weights = torch.tensor(compute_class_weight(class_weight="balanced", classes=np.unique(labels), y=labels))
        class_weights = torch.tensor([1, 2, 4])
        samples_weights = class_weights[labels]
#         print(class_weights)
        sampler = WeightedRandomSampler(weights=samples_weights, 
                                        num_samples=len(samples_weights), 
                                        replacement=True)

        dls = DataLoader(ds, 
                         batch_size=cfg['batch_size'], 
                         sampler=sampler, 
                         num_workers=os.cpu_count(), 
                         drop_last=True, 
                         pin_memory=True)
        
    elif split == 'valid' or split == 'test':
        eval_tsfm = tsfm['eval']
        ds = CustomDataset(data, transform=eval_tsfm)
        dls = DataLoader(ds, 
                         batch_size=2*cfg['batch_size'], 
                         shuffle=False, 
                         num_workers=os.cpu_count(), 
                         drop_last=False, 
                         pin_memory=True)
    else:
        raise Exception("Split should be 'train' or 'valid' or 'test'!!!")
    return dls

In [16]:
from sklearn import model_selection

kfold = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=2024)
x = train_data.index.values
y = train_data['target'].values.astype(int)
# g = train_data['series2id'].values.astype(int)

train_data['fold'] = -1
for fold, (tr_idx, val_idx) in enumerate(kfold.split(x,y)):
    train_data.loc[val_idx, 'fold'] = fold
    
train_data['fold'].value_counts()

fold
1    9732
0    9732
4    9731
3    9731
2    9731
Name: count, dtype: int64

In [17]:
FLIPS = [None, [-1], [-2], [-2, -1]]

def inference_loop(model, loader):
    model.to(CONFIG["device"])
    model.eval()
    preds = np.empty((0, 3))
    with torch.no_grad():
        for batch in tqdm(loader):
            images, labels = batch
            images = images.to(CONFIG["device"], non_blocking=True)
            with torch.autocast(device_type="cuda", dtype=torch.float16):
#                 logits = model(images.to(torch.float32))
                logits = model(images)
                logits = logits.softmax(dim=-1)
                preds = np.concatenate([preds, logits.detach().cpu().numpy()])
#     np.save('preds.npy', preds)
    return preds



def tta_inference_loop(model, loader):
    model.to(CONFIG["device"])
    model.eval()
    preds = np.empty((0, 3))
    with torch.no_grad():
        for batch in tqdm(loader):
            images, labels = batch
            images = images.to(CONFIG["device"], non_blocking=True)
            pred_tta = []
            with torch.autocast(device_type="cuda", dtype=torch.float16):
                for f in FLIPS:
                    logits = model(torch.flip(images, f) if f is not None else images)
                    logits = logits.softmax(dim=-1)
                    pred_tta.append(logits.detach().cpu().numpy())
#                 preds = np.concatenate([preds, logits.detach().cpu().numpy()])
                preds = np.concatenate([preds, np.mean(pred_tta, 0)])
#     np.save('preds.npy', preds)
    return preds

In [18]:
import wandb
run = wandb.init()
artifact = run.use_artifact('mandar4tech/RSNA-2024-Lumbar-Spine-Classification-Torch-RZoro/rsnaEffNetModel_0:v5', type='model')
artifact_dir = artifact.download()

wandb: Currently logged in as: mandar4tech. Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


In [19]:
# # checkpoints = glob.glob(f"{artifact_dir}/*.ckpt")[0]
# # lit_model2 = LumbarLightningModel2.load_from_checkpoint(checkpoints)
# # torch.save(lit_model2.model.state_dict(), "model_weights.pth")
# # weights_path = "/kaggle/working/model_weights.pth"
model = TimmModel(backbone=CONFIG["backbone"], pretrained=False)
weights_path = glob.glob(f"{artifact_dir}/*.pth")[0]
weights = torch.load(weights_path, map_location=torch.device("cpu"))
model.load_state_dict(weights)
torch.save(model.state_dict(), "model_weights.pth")